In [1]:
import pandas as pd
import numpy as np
np.random.seed(123)
from __future__ import print_function
import json
from pprint import pprint
import simplejson as json
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from collections import Counter
from operator import itemgetter
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline

In [2]:
with open('train.json', encoding='utf-8') as data_file:
    train = json.loads(data_file.read())
    #pprint(train)

In [3]:
with open('test.json', encoding='utf-8') as data_file:
    test = json.loads(data_file.read())

In [4]:
traindf=pd.DataFrame(train)
testdf=pd.DataFrame(test)

In [5]:
traindf['ingredients_string']= [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in traindf['ingredients']]
testdf['ingredients_string']= [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in testdf['ingredients']]

In [18]:
#traindf

In [6]:
traindf['cuisine'] = traindf['cuisine'].apply(lambda x: 0 if (x =="italian" or x=="mexican") else 1)

In [65]:
stwords=['pepper','salt','oil','garlic','fresh','ground','onion','black','chicken','chopped','tomatoes','red','powder'
          ,'tomato','flour','sauce','green','juice','dried','water','white','lime','butter','olive','cloves','white',
        'vegetable','chopped','egg','lemon','vinegar']

In [38]:
ytrain1 = traindf.cuisine
xtrain1= traindf.ingredients_string
xtest = testdf.ingredients_string
print(traindf.shape)
#xtrain1

(39774, 4)


In [39]:
xtrain=xtrain1.tail(15000)
ytrain=ytrain1.tail(15000)

In [40]:
xxtrain=xtrain1.head(4000)
yytrain=ytrain1.head(4000)

In [41]:
xtest=xtest.head(4000)

In [42]:
def tokens(x):
    return x.split('')

In [43]:
from sklearn.pipeline import Pipeline
from sklearn import model_selection
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
#vec=TfidfVectorizer(tokenizer=tokens,stop_words=('english'),
 #                          ngram_range = ( 1 , 1 ),analyzer="word", 
#                         max_df = .57 , binary=False , token_pattern=r'\w+' , sublinear_tf=False)
vec=CountVectorizer(stop_words=('english'))

In [44]:
# Learn vocabulary and idf, return term-document matrix.
xtrain=vec.fit_transform(xtrain).todense()
#Transform documents to document-term matrix.
xxtrain=vec.transform(xxtrain).todense()


In [45]:
xtest=vec.transform(xtest).todense()

In [46]:
print(xtrain.shape)

(15000, 2420)


In [47]:
print(xxtrain.shape)

(4000, 2420)


In [48]:
print(xtest.shape)

(4000, 2420)


In [49]:
#Logistic regression 
from sklearn import grid_search
from sklearn.linear_model import LogisticRegression
#cv = model_selection.KFold(3)
parameters = {'C':[1, 10]}
clf = LogisticRegression()
classifier_lr = grid_search.GridSearchCV(clf,parameters,cv=3,scoring='roc_auc')


In [50]:
classifier_lr.fit(xtrain,ytrain)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={'C': [1, 10]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [51]:
classifier_lr.score(xtrain,ytrain) #AUC: 0.999

C:\Users\aayya\Anaconda3\lib\site-packages\sklearn\grid_search.py:438: ChangedBehaviorWarning: The long-standing behavior to use the estimator's score function in GridSearchCV.score has changed. The scoring parameter is now used.
  ChangedBehaviorWarning)


0.98260749285834037

In [52]:
print(classifier_lr.best_estimator_)
# print(optimized_lr.cv_results_)

# evaluate on holdout
from sklearn.metrics import roc_auc_score
y_pred = classifier_lr.predict_proba(xxtrain)[:, 1]

roc_on_test = roc_auc_score(yytrain, y_pred)
print(roc_on_test)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
0.965628877313


In [53]:
#GBM
from sklearn import grid_search
parameters = {'n_estimators':[150],'max_features':[120],'learning_rate':[.2],'max_depth':[8]}
#learning_rate=[.2,.3,.4,.6,.7]
#n_estimators = [10]
#max_features = [5, 10]

from sklearn import ensemble
clf = ensemble.GradientBoostingClassifier()
classifier_GBM = grid_search.GridSearchCV(clf,parameters,cv=3,scoring='roc_auc')

In [54]:
classifier_GBM.fit(xtrain,ytrain)

GridSearchCV(cv=3, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'learning_rate': [0.2], 'max_features': [120], 'n_estimators': [150], 'max_depth': [8]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [56]:
classifier_GBM.best_score_

0.9647974367055026

In [55]:
classifier_GBM.score(xtrain,ytrain)

C:\Users\aayya\Anaconda3\lib\site-packages\sklearn\grid_search.py:438: ChangedBehaviorWarning: The long-standing behavior to use the estimator's score function in GridSearchCV.score has changed. The scoring parameter is now used.
  ChangedBehaviorWarning)


0.99343101317825533

In [57]:
print(classifier_GBM.best_estimator_)


# evaluate on text data(xxtrain and yytrain)
from sklearn.metrics import roc_auc_score
y_pred = classifier_GBM.predict_proba(xxtrain)[:, 1]

roc_on_test = roc_auc_score(yytrain, y_pred)
print(roc_on_test)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.2, loss='deviance', max_depth=8,
              max_features=120, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=150, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)
0.966297644054


In [58]:
#RF
from sklearn import grid_search
#from sklearn.linear_model import LogisticRegression
#cv = model_selection.KFold()
parameters = {'n_estimators':[50],'max_features':[15, 20, 25],'max_depth':[6]}
#estimators = [10, 20, 30, 40, 50]
#max_features = [5, 10, 15, 20, 25]
#max_depth = [3, 4, 5, 6]


from sklearn import ensemble
clf = ensemble.RandomForestClassifier(criterion='gini')
classifier_rf = grid_search.GridSearchCV(clf,parameters,cv=3,scoring='roc_auc')


In [59]:
classifier_rf.fit(xtrain,ytrain)

GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_features': [15, 20, 25], 'n_estimators': [50], 'max_depth': [6]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [62]:
print(classifier_rf.best_estimator_)
# print(optimized_lr.cv_results_)

# evaluate on holdout
from sklearn.metrics import roc_auc_score
y_pred = classifier_rf.predict_proba(xxtrain)[:, 1]

roc_on_test = roc_auc_score(yytrain, y_pred)
print(roc_on_test)

0.893146702305


In [60]:
classifier_rf.score(xtrain,ytrain)

C:\Users\aayya\Anaconda3\lib\site-packages\sklearn\grid_search.py:438: ChangedBehaviorWarning: The long-standing behavior to use the estimator's score function in GridSearchCV.score has changed. The scoring parameter is now used.
  ChangedBehaviorWarning)


0.91390366685822055

In [61]:
classifier_rf.best_score_

0.913404071281311

In [63]:
testdf=testdf.head(4000)

In [64]:
predictions=classifier_GBM.predict(xtest)

In [67]:
testdf['cuisine'] = predictions

In [68]:
#testdf = testdf.sort('id' , ascending=True)
testdf[['id' ,'ingredients_string', 'cuisine' ]].to_csv("submission1.csv")